# Decoding stimulus identity from neural responses

We implement here a nearest-neighbor decoder of neural activity.

In [1]:
# general modules
import pynwb, os, sys, pprint, itertools
import numpy as np
import matplotlib.pylab as plt

# custom modules
sys.path.append('..')
from physion.dataviz.show_data import MultimodalData, EpisodeResponse
from datavyz import graph_env_manuscript as ge

In [2]:
filename = '/home/yann.zerlaut/DATA/2022_03_16-16-21-18.nwb'
episodes = EpisodeResponse(filename, quantities=['dFoF', 'rawFluo', 'pupil'], 
                           verbose=True)

NWB-file reading time: 190.8ms
  Number of episodes over the whole recording: 40/40 (with protocol condition)
  building episodes with 3 modalities [...]

calculating dF/F with method "maximin" [...]

  ** 66 ROIs were discarded with the positive F0 criterion (42.6%) ** 

-> dFoF calculus done !  (calculation took 0.1s)
  -> [ok] episodes ready !


In [3]:
print('\n -* Varied Parameters *- \n')
pprint.pprint(episodes.varied_parameters)
VP = [key for key in episodes.varied_parameters if key!='repeat'] # varied parameters except rpeat
print(VP)


 -* Varied Parameters *- 

{'Image-ID': array([3., 4.]),
 'VSE-seed': array([3., 4.]),
 'repeat': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)}
['Image-ID', 'VSE-seed']


In [4]:
SUMMARY = {'significant_rois':[[] for i in range(np.product([len(episodes.varied_parameters[key]) for key in VP]))],
           'stim_pair':[[0,0], [0,1], [1,0], [1,1]]}

duration = episodes.data.metadata['presentation-duration']
value_threshold = .6
# looping over neurons
for roi in range(episodes.data.nROIs):
    roi_summary = episodes.compute_summary_data(dict(interval_pre=[-2,0], 
                                                      interval_post=[0,duration],
                                                      test='wilcoxon', 
                                                      positive=True),
                                                 response_args={'quantity':'dFoF', 
                                                                'roiIndex':roi},
                                               response_significance_threshold=0.01)
    for istim in range(len(roi_summary['significant'])):
        if roi_summary['significant'][istim] and (roi_summary['value'][istim]>value_threshold):
            SUMMARY['significant_rois'][istim].append(roi)           
            
#print(SUMMARY['significant_rois'])
#print(roi_summary)


In [ ]:
stim_index = 3
np.random.seed(6)
fig = episodes.plot_evoked_pattern(episodes.find_episode_cond(VP, SUMMARY['stim_pair'][stim_index]),
                                   rois=np.random.choice(SUMMARY['significant_rois'][stim_index], 7),
                                   quantity='dFoF')

In [ ]:
fig, AX = ge.figure(axes_extents=[[[1,3] for i in range(4)],
                                  [[1,9] for i in range(4)]], 
                     figsize=(1.3, .3), left=0.3,
                     top=12, right=3, wspace=1.2)
 

for stim_index in range(4):
    np.random.seed(6)
    episodes.plot_evoked_pattern(episodes.find_episode_cond(VP, SUMMARY['stim_pair'][stim_index]),
                                   rois=np.random.choice(SUMMARY['significant_rois'][stim_index], 8, replace=False),
                                   quantity='dFoF',
                                 axR=AX[0][stim_index], axT=AX[1][stim_index])
ge.save_on_desktop(fig, 'fig.png')